In [3]:
%matplotlib widget

import numpy as np
from bmcs_cross_section.api import ReinfLayer, BarLayer, MKappa, TShape
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import sympy as sp
import matplotlib as plt
import matplotlib.image as mpimg
from IPython.display import Image
from doepy import build


In [4]:

def plot_geometries(geometries, name="3D Geometries"):
    """
    Plots multiple 3D geometries in a single k3d plot and labels the nodes.

    Parameters:
        geometries (list of tuples): Each tuple contains:
            - positions (numpy.ndarray): (N,3) array of vertex coordinates.
            - indices (numpy.ndarray): (M,3) array defining mesh faces.
            - color (int): Hex color for the geometry.
        name (str, optional): Name of the plot window.

    Returns:
        k3d.Plot: The k3d plot object.
    """
    plot = k3d.plot(name=name)

    for idx, (positions, indices, color) in enumerate(geometries):
        # Create the points object
        plt_points = k3d.points(positions=positions.astype(np.float32),
                                shader='3d',
                                color=color, 
                                point_size=30)
        
        # Create the mesh object
        plt_mesh = k3d.mesh(vertices=positions, 
                            indices=indices, 
                            color=color,  
                            opacity=0.5)
        
        # Add elements to the plot
        plot += plt_points
        plot += plt_mesh
        
        # # Add labels for the nodes with slight offset
        # for i, position in enumerate(positions):
        #     offset = np.array([0, 25 * (idx + 1), 0])  # Move text slightly in the z-direction
        #     label_position = position + offset
            
        #     label = k3d.text(
        #         text=str(i),  # Label is the index of the node
        #         position=label_position.astype(np.float32),
        #         color=color,
        #         size=1  # Reduce label size to prevent clutter
        #     )
        #     plot += label

    return plot


In [5]:
t_num = 50
a_num= 1000/4
b_num= 1615/4
c_num= 645/4
e_num= 286/4
gamma_num= 0.683
reinf_layer_num = 2
wb_shell = WBTessellation4PEx(
                            a=a_num,
                            b = b_num, 
                            c = c_num, 
                            e_x = e_num,
                            gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                            n_phi_plus=2, # planned 5 
                            n_x_plus=2,  # planned 3
                            wireframe_width=5,
                            ##---- Trimming function works only in WBTessellation4P ----##
                            trim_half_cells_along_y=True,
                            trim_half_cells_along_x=True,
    #                          align_outer_nodes_along_x=True,
    )

orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

In [6]:


# Define symbols
a, b, c, e, gamma, t= sp.symbols(r'a b c e \gamma t', real=True)

# calculate the coordinates for folded state
x_folded = a+e
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)

alpha = sp.pi/2 - gamma

# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
}

In [7]:
O_gamma_lower = sp.Matrix([0, 0])
C_gamma_lower = sp.Matrix([e, 0])
V_gamma_lower = sp.Matrix([e+c*sp.sin(gamma), c*sp.cos(gamma)])
E_gamma_lower = sp.Matrix([a+e+c*sp.sin(gamma), c*sp.cos(gamma)])

In [8]:
O_gamma_upper = sp.Matrix([0, t])
C_gamma_upper = sp.Matrix([e-t*sp.tan(alpha/2), t])
V_gamma_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), c*sp.cos(gamma)+t])
E_gamma_upper = sp.Matrix([a+e+c*sp.sin(gamma), c*sp.cos(gamma)+t])

In [9]:
O_gamma_lower_num = np.array(O_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_lower_num = np.array(C_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_lower_num = np.array(V_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
E_gamma_lower_num = np.array(E_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
O_gamma_upper_num = np.array(O_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_gamma_upper_num = np.array(C_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_gamma_upper_num = np.array(V_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
E_gamma_upper_num = np.array(E_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
O_gamma_lower_num, C_gamma_lower_num, V_gamma_lower_num, E_gamma_lower_num, O_gamma_upper_num, C_gamma_upper_num, V_gamma_upper_num, E_gamma_upper_num


## M-kappa tool
mc = MKappa(low_kappa=0, high_kappa=0.0001)
mc.cs_design.cross_section_shape = 'custom'
mc.cs_design.cross_section_shape_.cs_points = np.array([
O_gamma_lower_num, 
C_gamma_lower_num, 
V_gamma_lower_num, 
E_gamma_lower_num, 
E_gamma_upper_num,
V_gamma_upper_num, 
C_gamma_upper_num, 
O_gamma_upper_num, 
])

mc.cs_design.concrete.compression_.trait_set(
        f_c = 68.2, eps_cy=-0.0035, eps_cu=-0.0038, E_cc=24805
    )

A_roving = (1.8+1.95)/2 # 3.62 # [mm**2] # Average of the range given on the catalogue
spacing = 12.7 # [mm] 
width = e_num # [mm] 
E_carbon = 200000 # [MPa] 
n_rovings = width / spacing
A_layer = n_rovings * A_roving
n_rovings, A_layer

_f_t = 1200
def add_carbon_layers(mc, num_layers, A_layer, E_carbon, _f_t, spacing=5):
    for i in range(num_layers):
        z = spacing * (i + 1)
        layer = ReinfLayer(name=f'carbon {i+1}', matmod='carbon', z=z, A=A_layer)
        layer.matmod_.trait_set(f_t=_f_t, E=E_carbon)
        mc.cross_section_layout.add_layer(layer)
        
add_carbon_layers(mc, num_layers=reinf_layer_num, A_layer=A_layer, E_carbon=E_carbon, _f_t=_f_t)     

# Multiply it by 2 to get the moment for the whole cross section, as only the half is considered in the analysis
M_R_kappa = abs(mc.M_t.max())*10**-6*2

output = mc._get_stresses_at_maxium_moment()
sig_reinf_t = output[0].max() # extract from array
sig_conc_c = output[1]
sig_conc_t = output[2]

wb_shell.interact()     
mc.high_kappa=0.0003
mc.interact()
mc.cs_design.cross_section_shape_.interact()

/Users/gui/Documents/VSCode/bmcs_env/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


in update plot 1e-07


In [10]:
M_R_kappa

np.float64(8.106610634894581)

In [11]:

import numpy as np
import pandas as pd

Lx_folded_list = []
Ly_folded_list = []
Lz_folded_list = []

# Results lists
M_R_kappa_list = []
sig_reinf_t_list, sig_conc_c_list, sig_conc_t_list = [], [], []

# Parameter ranges
a_list = list(np.arange(200, 500, 100))
b_list = list(np.arange(200, 500,100))
c_list = list(np.arange(100, 200, 50))
e_list = list(np.arange(40, 160, 50))
gamma_list = list(np.arange(0.4, 1.2, 0.25).astype(np.float32))
t_list = list(np.arange(15, 40, 10))
reinf_layer_list = list(np.arange(1, 6, 1))

# Create full factorial design
df = build.full_fact({
    'a': a_list,
    'b': b_list,
    'c': c_list,
    'e': e_list,
    'gamma': gamma_list,
    'thickness': t_list,
    'reinf_layer': reinf_layer_list
})

# Flat state geometry
Lx_flat = 2 * (df['a'] + df['c'] + df['e']) 
Ly_flat = 2 * df['b'] 
A = Lx_flat * Ly_flat
V_total = A * df['thickness'] 

A_roving_x = (1.8+1.95)/2 # [mm**2] # Average of the range given on the catalogue
A_roving_y = 0.45 # [mm**2] # Average of the range given on the catalogue

spacing_x = 12.7 # [mm] 
spacing_y = 16 # [mm]

n_rovings_x = Lx_flat / spacing_x
n_rovings_y = Ly_flat / spacing_y

V_reinf_x = n_rovings_x * A_roving_x * Ly_flat
V_reinf_y = n_rovings_y * A_roving_y * Lx_flat
V_reinf = (V_reinf_x + V_reinf_y) * df['reinf_layer'] # [mm**3]

V_conc = V_total - V_reinf

rho = (1.74 + 1.85) / 2 # [g/cm3] # Average of the range given on the catalogue
weight_reinf = V_reinf * rho # [g/cm3]
# Convert to kg/m3
weight_reinf = weight_reinf * 1e-9 # [kg/m3]

df['eta'] = df['b'] / df['a']
df['zeta'] = df['c'] / df['a']
df['Lx_flat mm'] = Lx_flat
df['Ly_flat mm'] = Ly_flat
df['area m2'] = A * 1e-6
df['volume_total m3'] = V_total * 1e-9
df['volume_concrete m3'] = V_conc * 1e-9
df['volume_reinforcement m3'] = V_reinf * 1e-9
df['weight_reinforcement kg'] = weight_reinf

# Filter based on thickness and reinforcement combinations
df = df[((df['thickness'] == 15) & (df['reinf_layer'].isin([2, 3]))) |
        ((df['thickness'] == 25) & (df['reinf_layer'].isin([3, 4]))) |
        ((df['thickness'] == 35) & (df['reinf_layer'].isin([4, 5])))]
df = df.reset_index(drop=True)

# ---------- Apply Constraint 1 ----------
# b^2 ≥ a^2 * (tan(gamma) - 1/cos(gamma))^2
constraint_1_rhs = (np.tan(df['gamma']) - 1 / np.cos(df['gamma'])) ** 2 * df['a'] ** 2
df = df[df['b'] ** 2 >= constraint_1_rhs]

# ---------- Apply Constraint 2 ----------
# |a tan(gamma) - a / cos(gamma) + c cos(gamma)| ≤ sqrt(2ac sin(gamma) - 2ac + b^2 + c^2 cos^2(gamma))
lhs = np.abs(df['a'] * np.tan(df['gamma']) - df['a'] / np.cos(df['gamma']) + df['c'] * np.cos(df['gamma']))
rhs = np.sqrt(2 * df['a'] * df['c'] * np.sin(df['gamma']) - 2 * df['a'] * df['c'] +
              df['b'] ** 2 + df['c'] ** 2 * np.cos(df['gamma']) ** 2)

df = df[lhs <= rhs]

# Final reset
df = df.reset_index(drop=True)

/Users/gui/Documents/VSCode/bmcs_env/lib/python3.12/site-packages/doepy/doe_functions.py:22: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.iloc[i][j]=r[j][int(df.iloc[i][j])]


In [12]:
df

,a,b,c,e,gamma,thickness,reinf_layer,eta,zeta,Lx_flat mm,Ly_flat mm,area m2,volume_total m3,volume_concrete m3,volume_reinforcement m3,weight_reinforcement kg
0,200.0,200.0,100.0,40.0,0.40,15.0,2.0,1.000000,0.500000,680.0,400.0,0.272,0.00408,0.003984,0.000096,0.000172
1,300.0,200.0,100.0,40.0,0.40,15.0,2.0,0.666667,0.333333,880.0,400.0,0.352,0.00528,0.005156,0.000124,0.000222
2,200.0,300.0,100.0,40.0,0.40,15.0,2.0,1.500000,0.500000,680.0,600.0,0.408,0.00612,0.005977,0.000143,0.000257
3,300.0,300.0,100.0,40.0,0.40,15.0,2.0,1.000000,0.333333,880.0,600.0,0.528,0.00792,0.007734,0.000186,0.000333
4,400.0,300.0,100.0,40.0,0.40,15.0,2.0,0.750000,0.250000,1080.0,600.0,0.648,0.00972,0.009492,0.000228,0.000409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,300.0,300.0,150.0,140.0,1.15,35.0,5.0,1.000000,0.500000,1180.0,600.0,0.708,0.02478,0.024158,0.000622,0.001117
1256,400.0,300.0,150.0,140.0,1.15,35.0,5.0,0.750000,0.375000,1380.0,600.0,0.828,0.02898,0.028252,0.000728,0.001306
1257,200.0,400.0,150.0,140.0,1.15,35.0,5.0,2.000000,0.750000,980.0,800.0,0.784,0.02744,0.026751,0.000689,0.001237
1258,300.0,400.0,150.0,140.0,1.15,35.0,5.0,1.333333,0.500000,1180.0,800.0,0.944,0.03304,0.032210,0.000830,0.001489


In [13]:
for i in range(len(df)):

    # Set the reference geometry to be generated from the closed form kinematics

    t_num = df['thickness'][i]
    a_num= df['a'][i]
    b_num= df['b'][i]
    c_num= df['c'][i]
    e_num= df['e'][i]
    gamma_num= df['gamma'][i]
    reinf_layer_num = df['reinf_layer'][i].astype(np.int32)

    wb_shell = WBTessellation4PEx(
                            a=a_num,
                            b = b_num, 
                            c = c_num, 
                            e_x = e_num,
                            gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                            n_phi_plus=2, # planned 5 
                            n_x_plus=2,  # planned 3
                            wireframe_width=5,
                            ##---- Trimming function works only in WBTessellation4P ----##
                            trim_half_cells_along_y=True,
                            trim_half_cells_along_x=True,
    #                          align_outer_nodes_along_x=True,
    )

    orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
    orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

    Lx_folded = orig_X_Ia[15][0] - orig_X_Ia[10][0]
    Ly_folded = orig_X_Ia[13][1] - orig_X_Ia[16][1]
    Lz_folded = (orig_X_Ia[6][2] - orig_X_Ia[0][2])+2*t_num

    Lx_folded_list.append(Lx_folded)
    Ly_folded_list.append(Ly_folded)
    Lz_folded_list.append(Lz_folded)



    # Define symbols
    a, b, c, e, gamma, t= sp.symbols(r'a b c e \gamma t', real=True)

    # calculate the coordinates for folded state
    x_folded = a+e
    y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
    z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)

    alpha = sp.pi/2 - gamma

    # Substitute parameters
    subs_dict = {
        a: a_num,
        b: b_num,
        c: c_num,
        e: e_num,
        gamma: gamma_num,
        t: t_num,
    }

    O_gamma_lower = sp.Matrix([0, 0])
    C_gamma_lower = sp.Matrix([e, 0])
    V_gamma_lower = sp.Matrix([e+c*sp.sin(gamma), c*sp.cos(gamma)])
    E_gamma_lower = sp.Matrix([a+e+c*sp.sin(gamma), c*sp.cos(gamma)])

    O_gamma_upper = sp.Matrix([0, t])
    C_gamma_upper = sp.Matrix([e-t*sp.tan(alpha/2), t])
    V_gamma_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), c*sp.cos(gamma)+t])
    E_gamma_upper = sp.Matrix([a+e+c*sp.sin(gamma), c*sp.cos(gamma)+t])


    O_gamma_lower_num = np.array(O_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
    C_gamma_lower_num = np.array(C_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
    V_gamma_lower_num = np.array(V_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
    E_gamma_lower_num = np.array(E_gamma_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
    O_gamma_upper_num = np.array(O_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
    C_gamma_upper_num = np.array(C_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
    V_gamma_upper_num = np.array(V_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
    E_gamma_upper_num = np.array(E_gamma_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
    O_gamma_lower_num, C_gamma_lower_num, V_gamma_lower_num, E_gamma_lower_num, O_gamma_upper_num, C_gamma_upper_num, V_gamma_upper_num, E_gamma_upper_num


    ## M-kappa tool
    mc = MKappa(low_kappa=0, high_kappa=0.0001)
    mc.cs_design.cross_section_shape = 'custom'
    mc.cs_design.cross_section_shape_.cs_points = np.array([
    O_gamma_lower_num, 
    C_gamma_lower_num, 
    V_gamma_lower_num, 
    E_gamma_lower_num, 
    E_gamma_upper_num,
    V_gamma_upper_num, 
    C_gamma_upper_num, 
    O_gamma_upper_num, 
    ])

    mc.cs_design.concrete.compression_.trait_set(
        f_c = 68.2, eps_cy=-0.0035, eps_cu=-0.0038, E_cc=24805
    )

    A_roving = (1.8+1.95)/2 # 3.62 # [mm**2] # Average of the range given on the catalogue
    spacing = 12.7 # [mm] 
    width = e_num # [mm] 
    E_carbon = 200000 # [MPa] 
    n_rovings = width / spacing
    A_layer = n_rovings * A_roving
    n_rovings, A_layer

    _f_t = 1200
    def add_carbon_layers(mc, num_layers, A_layer, E_carbon, _f_t, spacing=5):
        for i in range(num_layers):
            z = spacing * (i + 1)
            layer = ReinfLayer(name=f'carbon {i+1}', matmod='carbon', z=z, A=A_layer)
            layer.matmod_.trait_set(f_t=_f_t, E=E_carbon)
            mc.cross_section_layout.add_layer(layer)
            
    add_carbon_layers(mc, num_layers=reinf_layer_num, A_layer=A_layer, E_carbon=E_carbon, _f_t=_f_t)     

    # Multiply it by 2 to get the moment for the whole cross section, as only the half is considered in the analysis
    M_R_kappa = abs(mc.M_t.max())*10**-6*2
    M_R_kappa_list.append(M_R_kappa)

    output = mc._get_stresses_at_maxium_moment()
    sig_reinf_t = output[0].max() # extract from array
    sig_conc_c = output[1]
    sig_conc_t = output[2]

    sig_reinf_t_list.append(sig_reinf_t)
    sig_conc_c_list.append(sig_conc_c)
    sig_conc_t_list.append(sig_conc_t)



    #wb_shell.interact()     
    #mc.high_kappa=0.0003
    #mc.interact()
    #mc.cs_design.cross_section_shape_.interact()
    print(i)

df['Lx_folded'] = Lx_folded
df['Ly_folded'] = Ly_folded
df['Lz_folded'] = Lz_folded
df['flexural_capacity'] = M_R_kappa_list
df['sig_reinf_t'] = sig_reinf_t_list
df['sig_conc_c'] = sig_conc_c_list
df['sig_conc_t'] = sig_conc_t_list

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
df

,a,b,c,e,gamma,thickness,reinf_layer,eta,zeta,Lx_flat mm,...,volume_concrete m3,volume_reinforcement m3,weight_reinforcement kg,Lx_folded,Ly_folded,Lz_folded,flexural_capacity,sig_reinf_t,sig_conc_c,sig_conc_t
0,200.0,200.0,100.0,40.0,0.40,15.0,2.0,1.000000,0.500000,680.0,...,0.003984,0.000096,0.000172,1633.829171,796.633667,131.272948,2.663600,1169.923919,-15.379342,2.107112
1,300.0,200.0,100.0,40.0,0.40,15.0,2.0,0.666667,0.333333,880.0,...,0.005156,0.000124,0.000222,1633.829171,796.633667,131.272948,2.716303,1196.052367,-13.026709,2.347994
2,200.0,300.0,100.0,40.0,0.40,15.0,2.0,1.500000,0.500000,680.0,...,0.005977,0.000143,0.000257,1633.829171,796.633667,131.272948,2.663600,1169.923919,-15.379342,2.107112
3,300.0,300.0,100.0,40.0,0.40,15.0,2.0,1.000000,0.333333,880.0,...,0.007734,0.000186,0.000333,1633.829171,796.633667,131.272948,2.716303,1196.052367,-13.026709,2.347994
4,400.0,300.0,100.0,40.0,0.40,15.0,2.0,0.750000,0.250000,1080.0,...,0.009492,0.000228,0.000409,1633.829171,796.633667,131.272948,2.725461,1171.243807,-11.795956,3.538805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,300.0,300.0,150.0,140.0,1.15,35.0,5.0,1.000000,0.500000,1180.0,...,0.024158,0.000622,0.001117,1633.829171,796.633667,131.272948,16.502097,1195.357407,-37.998746,1.828728
1256,400.0,300.0,150.0,140.0,1.15,35.0,5.0,0.750000,0.375000,1380.0,...,0.028252,0.000728,0.001306,1633.829171,796.633667,131.272948,16.717901,1185.679042,-33.032990,3.391590
1257,200.0,400.0,150.0,140.0,1.15,35.0,5.0,2.000000,0.750000,980.0,...,0.026751,0.000689,0.001237,1633.829171,796.633667,131.272948,16.162835,1192.698898,-46.309914,2.100297
1258,300.0,400.0,150.0,140.0,1.15,35.0,5.0,1.333333,0.500000,1180.0,...,0.032210,0.000830,0.001489,1633.829171,796.633667,131.272948,16.502097,1195.357407,-37.998746,1.828728


In [15]:
df.to_excel('wb-cross-section-analysis.xlsx', index=False)

In [16]:
import pandas as pd
df = pd.read_excel('wb-cross-section-analysis.xlsx')
df

,a,b,c,e,gamma,thickness,reinf_layer,eta,zeta,Lx_flat mm,...,volume_concrete m3,volume_reinforcement m3,weight_reinforcement kg,Lx_folded,Ly_folded,Lz_folded,flexural_capacity,sig_reinf_t,sig_conc_c,sig_conc_t
0,200,200,100,40,0.40,15,2,1.000000,0.500000,680,...,0.003984,0.000096,0.000172,1633.829171,796.633667,131.272948,2.663600,1169.923919,-15.379342,2.107112
1,300,200,100,40,0.40,15,2,0.666667,0.333333,880,...,0.005156,0.000124,0.000222,1633.829171,796.633667,131.272948,2.716303,1196.052367,-13.026709,2.347994
2,200,300,100,40,0.40,15,2,1.500000,0.500000,680,...,0.005977,0.000143,0.000257,1633.829171,796.633667,131.272948,2.663600,1169.923919,-15.379342,2.107112
3,300,300,100,40,0.40,15,2,1.000000,0.333333,880,...,0.007734,0.000186,0.000333,1633.829171,796.633667,131.272948,2.716303,1196.052367,-13.026709,2.347994
4,400,300,100,40,0.40,15,2,0.750000,0.250000,1080,...,0.009492,0.000228,0.000409,1633.829171,796.633667,131.272948,2.725461,1171.243807,-11.795956,3.538805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,300,300,150,140,1.15,35,5,1.000000,0.500000,1180,...,0.024158,0.000622,0.001117,1633.829171,796.633667,131.272948,16.502097,1195.357407,-37.998746,1.828728
1256,400,300,150,140,1.15,35,5,0.750000,0.375000,1380,...,0.028252,0.000728,0.001306,1633.829171,796.633667,131.272948,16.717901,1185.679042,-33.032990,3.391590
1257,200,400,150,140,1.15,35,5,2.000000,0.750000,980,...,0.026751,0.000689,0.001237,1633.829171,796.633667,131.272948,16.162835,1192.698898,-46.309914,2.100297
1258,300,400,150,140,1.15,35,5,1.333333,0.500000,1180,...,0.032210,0.000830,0.001489,1633.829171,796.633667,131.272948,16.502097,1195.357407,-37.998746,1.828728


In [17]:
# extract a randomly 30 rows from the dataframe, but they need to be statistic significanbt
df = df.sample(n=30, random_state=1)
df = df.reset_index(drop=True)
df
df.to_excel('wb-cross-section-analysis-random.xlsx', index=False)
df

,a,b,c,e,gamma,thickness,reinf_layer,eta,zeta,Lx_flat mm,...,volume_concrete m3,volume_reinforcement m3,weight_reinforcement kg,Lx_folded,Ly_folded,Lz_folded,flexural_capacity,sig_reinf_t,sig_conc_c,sig_conc_t
0,200,200,150,90,0.90,35,5,1.000000,0.750000,880,...,0.012011,0.000309,0.000555,1633.829171,796.633667,131.272948,15.552600,1188.376124,-31.821703,3.403471
1,200,400,150,140,0.40,15,3,2.000000,0.750000,980,...,0.011347,0.000413,0.000742,1633.829171,796.633667,131.272948,19.468195,1167.851863,-31.205620,2.940451
2,300,200,150,140,0.65,15,3,0.666667,0.500000,1180,...,0.006831,0.000249,0.000447,1633.829171,796.633667,131.272948,16.828369,1196.069569,-24.675274,2.469550
3,300,200,100,140,1.15,15,3,0.666667,0.333333,1080,...,0.006252,0.000228,0.000409,1633.829171,796.633667,131.272948,3.768192,802.955065,-26.543806,2.749711
4,200,200,150,90,0.65,15,2,1.000000,0.750000,880,...,0.005156,0.000124,0.000222,1633.829171,796.633667,131.272948,7.617477,1198.831871,-19.505162,3.349063
5,400,400,150,40,0.90,15,2,1.000000,0.375000,1180,...,0.013828,0.000332,0.000596,1633.829171,796.633667,131.272948,2.779039,1197.892006,-11.596522,3.550031
6,200,400,100,90,0.65,35,4,2.000000,0.500000,780,...,0.021401,0.000439,0.000787,1633.829171,796.633667,131.272948,11.349112,1184.361565,-30.348338,2.898791
7,400,400,100,90,0.40,35,5,1.000000,0.250000,1180,...,0.032210,0.000830,0.001489,1633.829171,796.633667,131.272948,15.695281,1179.008850,-22.892850,2.965791
8,300,200,150,90,0.65,25,3,0.666667,0.500000,1080,...,0.010572,0.000228,0.000409,1633.829171,796.633667,131.272948,11.929618,1174.342814,-19.644563,3.556038
9,200,200,150,40,0.90,35,5,1.000000,0.750000,780,...,0.010646,0.000274,0.000492,1633.829171,796.633667,131.272948,7.096083,1188.174137,-21.342412,2.880257


In [18]:
Lx_flat = 2 * (df['a'] + df['c'] + df['e']) 
Ly_flat = 2 * df['b'] 
A = Lx_flat * Ly_flat

# This is my code for the Schalung

The equation for the volume of concrete is $V_\mathrm{concrete} = a + b +c $

$f_{ck}$